## Experiment 5 - 25ml - Hyperparameter tuning

In [1]:
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.losses import MeanSquaredError as LossMSE
from keras.optimizers import Adam, SGD
from wandb.keras import WandbCallback
from keras.models import Sequential
import tensorflow as tf
import numpy as np
import wandb
import gc

2024-04-22 11:09:03.985536: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 11:09:04.100428: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 11:09:04.547525: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/baran-tomik/bin/miniconda3/envs/tf/lib/
2024-04-22 11:09:04.547602: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvin

In [2]:
training_data_path = 'data/'

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-04-22 11:09:04.984440: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 11:09:04.989375: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 11:09:04.989540: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xbarantomik. Use `wandb login --relogin` to force relogin


True

In [5]:
sweep_configuration = {
    "name": "arch8_hyperparameter_tuning_cont",
    "method": "random",
    "metric": {"goal": "minimize", "name": "val_loss"},
    "parameters": {
        "dense_layer_1" : {"values" : [256, 512]},
        "dense_layer_2" : {"values" : [128, 256, 512]},
        "dense_layer_3" : {"values" : [128, 256, 512]},
        "dense_layer_4" : {"values" : [128, 256, 512]},
        "dense_layer_5" : {"values" : [128, 256]},
        "dropout_1": {"values": [0, 0.2, 0.4]},
        "dropout_2": {"values": [0, 0.2, 0.4]},
        "dropout_3": {"values": [0, 0.2, 0.4]},
        "dropout_4": {"values": [0, 0.2, 0.4]},
        "dropout_5": {"values": [0, 0.2, 0.4]},
        "learning_rate": {"distribution": "uniform", "min": 1e-4, "max": 1e-1},
        "batch_size": {"values": [64]},
        "epochs": {"values": [20]},
        "optimizer": {"values": ["adam", "sgd"]},
        "activation_function": {"values": ["sigmoid", "relu"]}
    },
}

sweep_id = wandb.sweep(sweep=sweep_configuration, project="DP")

Create sweep with ID: 5yuqgfos
Sweep URL: https://wandb.ai/xbarantomik/dp/sweeps/5yuqgfos


In [6]:
try:
  train_data = np.load(training_data_path + 'train_data.npy')
  val_data = np.load(training_data_path + 'val_data.npy')
  train_target_ratings = np.load(training_data_path + 'train_target_ratings.npy')
  val_target_ratings = np.load(training_data_path + 'val_target_ratings.npy')

except OSError as e:
  print('\nERROR loading data')
  print(e)
  quit()

In [7]:
def create_model(config, train_data_shape=None):
    model = Sequential()
    model.add(Dense(config['dense_layer_1'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_1']))

    model.add(Flatten())

    model.add(Dense(config['dense_layer_2'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_2']))

    model.add(Dense(config['dense_layer_3'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_3']))

    model.add(Dense(config['dense_layer_4'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_4']))

    model.add(Dense(config['dense_layer_5'], input_shape=(train_data_shape,)))
    model.add(Activation(config['activation_function']))
    model.add(Dropout(config['dropout_5']))

    model.add(Dense(1))
    model.add(Activation('linear'))

    return model

In [8]:
wandb.config = {
  "dense_layer_1": 512,
  "dense_layer_2": 512,
  "dense_layer_3": 256,
  "dense_layer_4": 128,
  "dense_layer_5": 128,
  "dropout_1": 0.2,
  "dropout_2": 0.2,
  "dropout_3": 0.2,
  "dropout_4": 0.2,
  "dropout_5": 0.2,
  "learning_rate": 0.001,
  "batch_size": 128,
  "epochs": 20,
  "optimizer": "adam",
  "activation_function": "sigmoid",
}

In [9]:
def train():
    with wandb.init(
          project="DP",
          entity="xbarantomik",
          name="arch8_25m_added_imdb_context_max_abs_scaler_hyperparameter_tuning",
          group="hyperparameter_tuning"
        ) as run:
        config = wandb.config
        model = create_model(config, train_data_shape=train_data.shape[1])
        if config['optimizer'] == "sgd":
          model.compile(
            optimizer= SGD(
              learning_rate = config['learning_rate']
            ),
            loss=LossMSE(),
          )
        else:
          model.compile(
            optimizer= Adam(
              learning_rate = config['learning_rate']
            ),
            loss=LossMSE(),
          )
        gc.collect() 
        model.fit(train_data, train_target_ratings, validation_data=(val_data, val_target_ratings), callbacks=[WandbCallback()], verbose=1)

In [10]:
count = 25
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: lrfz5syi with config:
wandb: 	activation_function: sigmoid
wandb: 	batch_size: 64
wandb: 	dense_layer_1: 256
wandb: 	dense_layer_2: 256
wandb: 	dense_layer_3: 128
wandb: 	dense_layer_4: 128
wandb: 	dense_layer_5: 128
wandb: 	dropout_1: 0
wandb: 	dropout_2: 0
wandb: 	dropout_3: 0.2
wandb: 	dropout_4: 0
wandb: 	dropout_5: 0.2
wandb: 	epochs: 20
wandb: 	learning_rate: 0.03624794177455191
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


2024-04-22 11:09:10.953432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 11:09:10.953962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 11:09:10.954163: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-22 11:09:10.954282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

624567/624587 [============================>.] - ETA: 0s - loss: 1.1285INFO:tensorflow:Assets written to: /home/baran-tomik/projects/wandb_hpt/wandb/run-20240422_110909-lrfz5syi/files/model-best/assets


wandb: Adding directory to artifact (/home/baran-tomik/projects/wandb_hpt/wandb/run-20240422_110909-lrfz5syi/files/model-best)... Done. 0.0s


624587/624587 [==============================] - 993s 2ms/step - loss: 1.1285 - val_loss: 1.1348


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: vvbe1b6l with config:
wandb: 	activation_function: sigmoid
wandb: 	batch_size: 64
wandb: 	dense_layer_1: 512
wandb: 	dense_layer_2: 256
wandb: 	dense_layer_3: 256
wandb: 	dense_layer_4: 128
wandb: 	dense_layer_5: 256
wandb: 	dropout_1: 0
wandb: 	dropout_2: 0.2
wandb: 	dropout_3: 0.2
wandb: 	dropout_4: 0.4
wandb: 	dropout_5: 0
wandb: 	epochs: 20
wandb: 	learning_rate: 0.057223013811567934
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


624569/624587 [============================>.] - ETA: 0s - loss: 1.1296INFO:tensorflow:Assets written to: /home/baran-tomik/projects/wandb_hpt/wandb/run-20240422_180601-vvbe1b6l/files/model-best/assets


wandb: Adding directory to artifact (/home/baran-tomik/projects/wandb_hpt/wandb/run-20240422_180601-vvbe1b6l/files/model-best)... Done. 0.0s


624587/624587 [==============================] - 979s 2ms/step - loss: 1.1296 - val_loss: 1.1269


epoch,▁
loss,▁
val_loss,▁
best_epoch,0
best_val_loss,1.12694
epoch,0
loss,1.12961
val_loss,1.12694


In [11]:
gc.collect()

17205

: 